In [7]:
!pip install --upgrade sentence-transformers

  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transformers-4.1.0-py3-none-any.whl (345 kB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.13.4
    Uninstalling huggingface-hub-0.13.4:
      Successfully uninstalled huggingface-hub-0.13.4
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [8]:
!pip install --upgrade huggingface_hub


In [9]:
import torch
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")

    #Nvidia Geforce RTX 3070 Ti Laptop GPU

CUDA is available. Using GPU: NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [11]:
import pandas as pd
import huggingface_hub
from sentence_transformers import SentenceTransformer, util
import torch
import os

# Load INSTRUCTOR-Large model
model = SentenceTransformer("hkunlp/instructor-large")

#  Hotwords and instruction
hotwords = ["be careful", "destroy", "stranger"]
instruction = "Represent the intent of this sentence:"

# Input file
input_path = r"C:\Users\cheny\coderepo\airepo\cv-valid-dev.csv"

# Output directory and file
output_dir = r"C:\Users\cheny\coderepo\airepo\hotword-detection"
os.makedirs(output_dir, exist_ok=True)
output_filename = "cv-valid-dev_task5b.csv"
output_path = os.path.join(output_dir, output_filename)

# Load input CSV
df = pd.read_csv(input_path)
df['text'] = df['text'].astype(str)

# Encode hotwords
hotword_embeddings = model.encode(
    [[instruction, hw] for hw in hotwords],
    convert_to_tensor=True
)

# Encode all text entries
text_embeddings = model.encode(
    [[instruction, t] for t in df['text']],
    convert_to_tensor=True,
    show_progress_bar=True
)

# Compute similarity scores
similarity_scores = util.cos_sim(text_embeddings, hotword_embeddings)
max_similarities = torch.max(similarity_scores, dim=1).values

# Move to CPU and numpy before inserting into dataframe
SIMILARITY_THRESHOLD = 0.6
df['similarity'] = (max_similarities.cpu() >= SIMILARITY_THRESHOLD).numpy()

# Save updated CSV
df.to_csv(output_path, index=False)
print(f"Saved updated file with 'similarity' column to:\n{output_path}")




Batches:   0%|          | 0/128 [00:00<?, ?it/s]

Saved updated file with 'similarity' column to:
C:\Users\cheny\coderepo\airepo\hotword-detection\cv-valid-dev_task5b.csv
